In [1]:
library('DBI')
library('RPostgreSQL')
library('dbplyr')
library('dplyr')
library(tidyr)

# connect to database
con <- dbConnect(PostgreSQL(), host="localhost", port=5432, dbname="knock100", user="guest", password="guest")

# get access to table
re = tbl(con,'receipt')


Attaching package: ‘dplyr’


The following objects are masked from ‘package:dbplyr’:

    ident, sql


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




### R-041
レシート明細データ（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、前回売上があった日からの売上金額増減を計算せよ。そして結果を10件表示せよ。

In [57]:
re %>%
group_by(sales_ymd) %>%
summarise(sales_today = sum(amount)) %>%
mutate(sales_yest = sales_today %>% lag()) %>%
mutate(diff = sales_today - sales_yest) %>%
head

# Source:   lazy query [?? x 4]
# Database: postgres 12.0.11 [@localhost:5432/knock100]
  sales_ymd sales_today sales_yest   diff
      <int>       <dbl>      <dbl>  <dbl>
1  20190504       49561         NA     NA
2  20180414       27599      49561 -21962
3  20170416       16442      27599 -11157
4  20190422       37044      16442  20602
5  20190710       27796      37044  -9248
6  20181027       28497      27796    701

In [3]:
re %>%
group_by(sales_ymd) %>%
summarise(sales_today = sum(amount)) %>%
mutate(sales_yest = ifelse(is.na(lag(sales_today)),0,lag(sales_today))) %>%
mutate(diff = sales_today - sales_yest) %>%
head

Warning message:
“Missing values are always removed in SQL.
Use `SUM(x, na.rm = TRUE)` to silence this warning
This warning is displayed only once per session.”


# Source:   lazy query [?? x 4]
# Database: postgres 12.0.11 [@localhost:5432/knock100]
  sales_ymd sales_today sales_yest   diff
      <int>       <dbl>      <dbl>  <dbl>
1  20190504       49561          0  49561
2  20180414       27599      49561 -21962
3  20170416       16442      27599 -11157
4  20190422       37044      16442  20602
5  20190710       27796      37044  -9248
6  20181027       28497      27796    701

### R-042
レシート明細データ（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、各日付のデータに対し、前回、前々回、3回前に売上があった日のデータを結合せよ。そして結果を10件表示せよ。

In [4]:
re %>%
group_by(sales_ymd) %>%
summarise(sales_today = sum(amount)) %>%
mutate(sales_1 = ifelse(is.na(lag(sales_today,n=1)),0,lag(sales_today,n=1)),
		sales_2 = ifelse(is.na(lag(sales_today,n=2)),0,lag(sales_today,n=2)),
		sales_3 = ifelse(is.na(lag(sales_today,n=3)),0,lag(sales_today,n=3))
) %>%
head

# Source:   lazy query [?? x 5]
# Database: postgres 12.0.11 [@localhost:5432/knock100]
  sales_ymd sales_today sales_1 sales_2 sales_3
      <int>       <dbl>   <dbl>   <dbl>   <dbl>
1  20190504       49561       0       0       0
2  20180414       27599   49561       0       0
3  20170416       16442   27599   49561       0
4  20190422       37044   16442   27599   49561
5  20190710       27796   37044   16442   27599
6  20181027       28497   27796   37044   16442

In [55]:
re %>%
  group_by(sales_ymd) %>%
  summarise(sales_daily= sum(sales_ymd)) %>%
  mutate(
    across(
      sales_daily,
      list(lag_1 = ~ lag(.x, n = 1), lag_2 = ~ lag(.x, n = 2), lag_3 = ~ lag(.x, n = 3))
    )
  ) %>%
  head()

# Source:   lazy query [?? x 5]
# Database: postgres 12.0.11 [@localhost:5432/knock100]
   sales_ymd sales_daily sales_daily_lag_1 sales_daily_lag_2 sales_daily_lag_3
       <int>       <dbl>             <dbl>             <dbl>             <dbl>
 1  20190504  2382479472                NA                NA                NA
 2  20180414  2118943470        2382479472                NA                NA
 3  20170416  1593462864        2118943470        2382479472                NA
 4  20190422  2261327264        1593462864        2118943470        2382479472
 5  20190710  2059452420        2261327264        1593462864        2118943470
 6  20181027  1836473457        2059452420        2261327264        1593462864
 7  20180819  2280432547        1836473457        2059452420        2261327264
 8  20181107  2159378449        2280432547        1836473457        2059452420
 9  20170315  2057372130        2159378449        2280432547        1836473457
10  20170809  2037251709        2057372130 

### R-043:
レシート明細データ（df_receipt）と顧客データ（df_customer）を結合し、性別コード（gender_cd）と年代（ageから計算）ごとに売上金額（amount）を合計した売上サマリデータを作成せよ。性別コードは0が男性、1が女性、9が不明を表すものとする。

ただし、項目構成は年代、女性の売上金額、男性の売上金額、性別不明の売上金額の4項目とすること（縦に年代、横に性別のクロス集計）。また、年代は10歳ごとの階級とすること。

In [81]:
cu = tbl(con,'customer')
re %>%
inner_join(cu, by = 'customer_id') %>%
mutate(generation = age - age%%10) %>%
group_by(generation, gender_cd) %>%
summarise(sales_amount = amount %>% sum(), .groups = 'drop') %>% 
pivot_wider(names_from = gender_cd, names_prefix = 'sales', values_from = sales_amount, values_fill = 0) %>%
rename(sales_male = sales0, sales_female = sales1, sales_other = sales9) -> sales_summary
sales_summary 

# Source:   lazy query [?? x 4]
# Database: postgres 12.0.11 [@localhost:5432/knock100]
  generation sales_male sales_female sales_other
       <dbl>      <dbl>        <dbl>       <dbl>
1         10       1591       149836        4317
2         20      72940      1363724       44328
3         30     177322       693047       50441
4         40      19355      9320791      483512
5         50      54320      6685192      342923
6         60     272469       987741       71418
7         70      13435        29764        2427
8         80      46360       262923        5111
9         90          0         6260           0

### R-044
043で作成した売上サマリデータ（df_sales_summary）は性別の売上を横持ちさせたものであった。このデータから性別を縦持ちさせ、年代、性別コード、売上金額の3項目に変換せよ。ただし、性別コードは男性を"00"、女性を"01"、不明を"99"とする

In [85]:
sales_summary %>%
rename('00'=sales_male, '01'=sales_female, '99'=sales_other) %>%
  pivot_longer(cols = - generation, names_to = 'gender_cd', values_to = 'sales_amount') %>% head

# Source:   lazy query [?? x 3]
# Database: postgres 12.0.11 [@localhost:5432/knock100]
  generation gender_cd sales_amount
       <dbl> <chr>            <dbl>
1         10 00                1591
2         20 00               72940
3         30 00              177322
4         40 00               19355
5         50 00               54320
6         60 00              272469

In [6]:
sales_summary %>%
select(
	generation,
	'01' = sales_female,
	'00' = sales_male,
	'99' = sales_unknown
	) %>%
pivot_longer(!generation, names_to = 'gender', values_to = 'sales') %>%
head(10)

# Source:   lazy query [?? x 3]
# Database: postgres 12.0.11 [@localhost:5432/knock100]
   generation gender   sales
        <dbl> <chr>    <dbl>
 1         10 01      149836
 2         20 01     1363724
 3         30 01      693047
 4         40 01     9320791
 5         50 01     6685192
 6         60 01      987741
 7         70 01       29764
 8         80 01      262923
 9         90 01        6260
10         10 00        1591

### R-045
顧客データ（df_customer）の生年月日（birth_day）は日付型でデータを保有している。これをYYYYMMDD形式の文字列に変換し、顧客ID（customer_id）とともに10件表示せよ。

In [48]:
cu %>%
transmute(customer_id ,date = birth_day %>% as.character() %>% str_replace_all(pattern = '-',replacement='')) %>%
head

# Source:   lazy query [?? x 2]
# Database: postgres 12.0.11 [@localhost:5432/knock100]
  customer_id    date    
  <chr>          <chr>   
1 CS021313000114 19810429
2 CS037613000071 19520401
3 CS031415000172 19761004
4 CS028811000001 19330327
5 CS001215000145 19950329
6 CS020401000016 19740915

### R-046
顧客データ（df_customer）の申し込み日（application_date）はYYYYMMDD形式の文字列型でデータを保有している。これを日付型に変換し、顧客ID（customer_id）とともに10件表示せよ。

In [8]:
cu %>%
transmute(customer_id, application_date = as.Date(application_date)) %>%
head

# Source:   lazy query [?? x 2]
# Database: postgres 12.0.11 [@localhost:5432/knock100]
  customer_id    application_date
  <chr>          <date>          
1 CS021313000114 2015-09-05      
2 CS037613000071 2015-04-14      
3 CS031415000172 2015-05-29      
4 CS028811000001 2016-01-15      
5 CS001215000145 2017-06-05      
6 CS020401000016 2015-02-25      

### R-047
レシート明細データ（df_receipt）の売上日（sales_ymd）はYYYYMMDD形式の数値型でデータを保有している。これを日付型に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。

In [47]:
re %>%
transmute(sales_date = sales_ymd %>% as.character() %>% as.Date(),
			receipt_no,receipt_sub_no) %>%
head

# Source:   lazy query [?? x 3]
# Database: postgres 12.0.11 [@localhost:5432/knock100]
  sales_date receipt_no receipt_sub_no
  <date>          <int>          <int>
1 2018-11-03        112              1
2 2018-11-18       1132              2
3 2017-07-12       1102              1
4 2019-02-05       1132              1
5 2018-08-21       1102              2
6 2019-06-05       1112              1

### R-052:
レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計の上、売上金額合計に対して2,000円以下を0、2,000円より大きい金額を1に二値化し、顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [40]:
library(stringr)
re %>%
filter(!str_detect(customer_id, '^Z')) %>%
group_by(customer_id) %>%
summarise(amount = sum(amount, na.rm = TRUE)) %>%
mutate(amount_over_2000 = ifelse(amount>2000,1,0)) %>%
head(10)

# Source:   lazy query [?? x 3]
# Database: postgres 12.0.11 [@localhost:5432/knock100]
   customer_id    amount amount_over_2000
   <chr>           <dbl>            <dbl>
 1 CS001311000059   2302                1
 2 CS004614000122    248                0
 3 CS003512000043    298                0
 4 CS011615000061    246                0
 5 CS029212000033   3604                1
 6 CS007515000119   7157                1
 7 CS034515000123   3699                1
 8 CS004315000058    490                0
 9 CS026414000014   6671                1
10 CS001615000099    768                0

### R-053:
顧客データ（df_customer）の郵便番号（postal_cd）に対し、東京（先頭3桁が100〜209のもの）を1、それ以外のものを0に二値化せよ。さらにレシート明細データ（df_receipt）と結合し、全期間において売上実績のある顧客数を、作成した二値ごとにカウントせよ。

In [86]:
cu = tbl(con,'customer')

cu %>% mutate(is_tokyo = dplyr::between(substr(postal_cd,1,3) %>% as.integer(),100,209) %>% ifelse(1,0)) %>%
inner_join(re, by = 'customer_id') %>%
group_by(is_tokyo) %>%
summarise(amount=n_distinct(customer_id)) %>% head

# Source:   lazy query [?? x 2]
# Database: postgres 12.0.11 [@localhost:5432/knock100]
  is_tokyo amount
     <dbl>  <dbl>
1        0   3906
2        1   4400

### R-054:
顧客データ（df_customer）の住所（address）は、埼玉県、千葉県、東京都、神奈川県のいずれかとなっている。都道府県毎にコード値を作成し、顧客ID、住所とともに10件表示せよ。値は埼玉県を11、千葉県を12、東京都を13、神奈川県を14とすること。

In [12]:
cu %>%
mutate(prefecture_cd =
		case_when(
			substr(address,1,2) == '埼玉' ~ '11',
			substr(address,1,2) == '千葉' ~ '12',
			substr(address,1,2) == '東京' ~ '13',
			substr(address,1,3) == '神奈川' ~ '14'
		)) %>%
select(customer_id, address, prefecture_cd) %>%
head

# Source:   lazy query [?? x 3]
# Database: postgres 12.0.11 [@localhost:5432/knock100]
  customer_id    address                            prefecture_cd
  <chr>          <chr>                              <chr>        
1 CS021313000114 神奈川県伊勢原市粟窪**********     14           
2 CS037613000071 東京都江東区南砂**********         13           
3 CS031415000172 東京都渋谷区代々木**********       13           
4 CS028811000001 神奈川県横浜市泉区和泉町********** 14           
5 CS001215000145 東京都大田区仲六郷**********       13           
6 CS020401000016 東京都板橋区若木**********         13           

### R-055:
レシート明細（df_receipt）データの売上金額（amount）を顧客ID（customer_id）ごとに合計し、その合計金額の四分位点を求めよ。その上で、顧客ごとの売上金額合計に対して以下の基準でカテゴリ値を作成し、顧客ID、売上金額合計とともに10件表示せよ。カテゴリ値は順に1〜4とする。
- 最小値以上第1四分位未満 ・・・ 1を付与
- 第1四分位以上第2四分位未満 ・・・ 2を付与
- 第2四分位以上第3四分位未満 ・・・ 3を付与
- 第3四分位以上 ・・・ 4を付与

In [13]:
# assume data are in local machine
rel = DBI::dbReadTable(con, "receipt")
rel %>%
group_by(customer_id) %>%
summarise(amount = sum(amount, na.rm = TRUE)) %>%
mutate(pct_group = 
		case_when(
			amount < quantile(amount)[2] ~ 1
			,amount < quantile(amount)[3] ~ 2
			,amount < quantile(amount)[4] ~ 3
			,quantile(amount)[4] <= amount ~ 4
		)) %>%
head

customer_id,amount,pct_group
<chr>,<int>,<dbl>
CS001113000004,1298,2
CS001114000005,626,2
CS001115000010,3044,3
CS001205000004,1988,3
CS001205000006,3337,3
CS001211000025,456,1


In [ ]:
re %>%
group_by(customer_id) %>%
summarise(amount = sum(amount, na.rm = TRUE)) %>%
mutate(pct_group = 4) %>%
mutate_at(pct_group,function(x){})
head

### R-056:
顧客データ（df_customer）の年齢（age）をもとに10歳刻みで年代を算出し、顧客ID（customer_id）、生年月日（birth_day）とともに10件表示せよ。ただし、60歳以上は全て60歳代とすること。年代を表すカテゴリ名は任意とする。

In [90]:
cu %>%
mutate(generation = ifelse(age>=60, 60, age-age%%10)) -> customer_generation

customer_generation %>% 
select(customer_id, birth_day, generation) %>%
head

# Source:   lazy query [?? x 3]
# Database: postgres 12.0.11 [@localhost:5432/knock100]
  customer_id    birth_day  generation
  <chr>          <date>          <dbl>
1 CS021313000114 1981-04-29         30
2 CS037613000071 1952-04-01         60
3 CS031415000172 1976-10-04         40
4 CS028811000001 1933-03-27         60
5 CS001215000145 1995-03-29         20
6 CS020401000016 1974-09-15         40

### R-057:
056の抽出結果と性別コード（gender_cd）により、新たに性別×年代の組み合わせを表すカテゴリデータを作成し、10件表示せよ。組み合わせを表すカテゴリの値は任意とする。

In [91]:
generation %>%
mutate(gender_generation = gender_cd + generation) %>%
select(customer_id, gender_cd, gender, age, generation, gender_generation) %>%
head

# Source:   lazy query [?? x 6]
# Database: postgres 12.0.11 [@localhost:5432/knock100]
  customer_id    gender_cd gender   age generation gender_generation
  <chr>              <int> <chr>  <int>      <dbl>             <dbl>
1 CS021313000114         1 女性      37         30                31
2 CS037613000071         9 不明      66         60                69
3 CS031415000172         1 女性      42         40                41
4 CS028811000001         1 女性      86         60                61
5 CS001215000145         1 女性      24         20                21
6 CS020401000016         0 男性      44         40                40

### R-058:
顧客データ（df_customer）の性別コード（gender_cd）をダミー変数化し、顧客ID（customer_id）とともに10件表示せよ。

In [16]:
cul = DBI::dbReadTable(con, "customer")

library(recipes)
library(tidymodels)

cul %>%
  transmute(customer_id, gender_cd = factor(gender_cd)) %>%
  recipe() %>%
  step_dummy(gender_cd, one_hot = T) %>%
  prep() %>%
  juice() %>%
  head(10)

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”

Attaching package: ‘recipes’


The following object is masked from ‘package:stringr’:

    fixed


The following object is masked from ‘package:stats’:

    step


── Attaching packages ────────────────────────────────────── tidymodels 0.2.0 ──

✔ broom        0.8.0     ✔ rsample      0.1.1
✔ dials        0.1.1     ✔ tibble       3.1.7
✔ ggplot2      3.3.6     ✔ tune         0.2.0
✔ infer        1.0.0     ✔ workflows    0.2.6
✔ modeldata    0.1.1     ✔ workflowsets 0.2.1
✔ parsnip      0.2.1     ✔ yardstick    0.0.9
✔ purrr        0.3.4     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ purrr::discard() masks scales::discard()
✖ dplyr::filter()  masks stats::filter()
✖ recipes::fixed() masks stringr::fixed()
✖ dplyr::ident()   masks dbplyr::ident()
✖ dplyr::lag()     masks stats::lag()
✖ dplyr::sql()     masks dbplyr::sql()
✖ recipes::step()  mask

customer_id,gender_cd_X0,gender_cd_X1,gender_cd_X9
<fct>,<dbl>,<dbl>,<dbl>
CS021313000114,0,1,0
CS037613000071,0,0,1
CS031415000172,0,1,0
CS028811000001,0,1,0
CS001215000145,0,1,0
CS020401000016,1,0,0
CS015414000103,0,1,0
CS029403000008,1,0,0
CS015804000004,1,0,0


In [17]:
cu %>% 
transmute(customer_id, gender_cd = 
		case_when(
			gender_cd == 1~'female',
			gender_cd == 0~'male',
			gender_cd == 9~'unknown'
		)) %>%
mutate(name = gender_cd, value = 1L) %>%
pivot_wider(values_fill = 0L) %>%
head(10)

# Source:   lazy query [?? x 5]
# Database: postgres 12.0.11 [@localhost:5432/knock100]
   customer_id    gender_cd female unknown  male
   <chr>          <chr>      <int>   <int> <int>
 1 CS001105000001 male           0       0     1
 2 CS001112000009 female         1       0     0
 3 CS001112000019 female         1       0     0
 4 CS001112000021 female         1       0     0
 5 CS001112000023 female         1       0     0
 6 CS001112000024 female         1       0     0
 7 CS001112000029 female         1       0     0
 8 CS001112000030 female         1       0     0
 9 CS001113000004 female         1       0     0
10 CS001113000010 female         1       0     0

In [95]:
cu %>% select(customer_id, gender_cd) %>%
mutate(value = 1) %>%
pivot_wider(names_from = gender_cd, names_prefix = 'gender_cd' ,values_from = value, values_fill = 0) 

# Source:   lazy query [?? x 4]
# Database: postgres 12.0.11 [@localhost:5432/knock100]
   customer_id    gender_cd0 gender_cd9 gender_cd1
   <chr>               <dbl>      <dbl>      <dbl>
 1 CS001105000001          1          0          0
 2 CS001112000009          0          0          1
 3 CS001112000019          0          0          1
 4 CS001112000021          0          0          1
 5 CS001112000023          0          0          1
 6 CS001112000024          0          0          1
 7 CS001112000029          0          0          1
 8 CS001112000030          0          0          1
 9 CS001113000004          0          0          1
10 CS001113000010          0          0          1
# … with more rows

### R-059:
レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を平均0、標準偏差1に標準化して顧客ID、売上金額合計とともに10件表示せよ。標準化に使用する標準偏差は、分散の平方根、もしくは不偏分散の平方根のどちらでも良いものとする。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [96]:
library(stringr)
re %>%
filter(!str_detect(customer_id,'^Z')) %>%
group_by(customer_id) %>%
summarise(amount = sum(amount, na.rm = TRUE)) %>%
mutate(std_amount = scale(amount, center = TRUE, scale = TRUE)) %>%
head

Warning message:
“Named arguments ignored for SQL scale”
ERROR while rich displaying an object: Error in postgresqlExecStatement(conn, statement, ...): RS-DBI driver: (<af>OlU)

Traceback:
1. tryCatch(withCallingHandlers({
 .     if (!mime %in% names(repr::mime2repr)) 
 .         stop("No repr_* for mimetype ", mime, " in repr::mime2repr")
 .     rpr <- repr::mime2repr[[mime]](obj)
 .     if (is.null(rpr)) 
 .         return(NULL)
 .     prepare_content(is.raw(rpr), rpr)
 . }, error = error_handler), error = outer_handler)
2. tryCatchList(expr, classes, parentenv, handlers)
3. tryCatchOne(expr, names, parentenv, handlers[[1L]])
4. doTryCatch(return(expr), name, parentenv, handler)
5. withCallingHandlers({
 .     if (!mime %in% names(repr::mime2repr)) 
 .         stop("No repr_* for mimetype ", mime, " in repr::mime2repr")
 .     rpr <- repr::mime2repr[[mime]](obj)
 .     if (is.null(rpr)) 
 .         return(NULL)
 .     prepare_content(is.raw(rpr), rpr)
 . }, error = error_handler)
6

In [18]:
library(stringr)
re %>%
filter(!str_detect(customer_id,'^Z')) %>%
group_by(customer_id) %>%
summarise(amount = sum(amount, na.rm = TRUE)) %>%
mutate(std_amount = (amount - mean(amount))/sd(amount)) %>%
head

Warning message:
“Missing values are always removed in SQL.
Use `AVG(x, na.rm = TRUE)` to silence this warning
This warning is displayed only once per session.”
Warning message:
“Missing values are always removed in SQL.
Use `STDDEV_SAMP(x, na.rm = TRUE)` to silence this warning
This warning is displayed only once per session.”


# Source:   lazy query [?? x 3]
# Database: postgres 12.0.11 [@localhost:5432/knock100]
  customer_id    amount std_amount
  <chr>           <dbl>      <dbl>
1 CS001311000059   2302    -0.0903
2 CS004614000122    248    -0.845 
3 CS003512000043    298    -0.827 
4 CS011615000061    246    -0.846 
5 CS029212000033   3604     0.388 
6 CS007515000119   7157     1.69  

### R-060:
レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を最小値0、最大値1に正規化して顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [19]:
library(stringr)
re %>%
filter(!str_detect(customer_id, '^Z')) %>%
group_by(customer_id) %>%
summarise(amount = sum(amount, na.rm = TRUE )) %>%
mutate(relative_pos = as.numeric((amount - min(amount)))/(max(amount)-min(amount))) %>%
head(10)

Warning message:
“Missing values are always removed in SQL.
Use `MIN(x, na.rm = TRUE)` to silence this warning
This warning is displayed only once per session.”
Warning message:
“Missing values are always removed in SQL.
Use `MAX(x, na.rm = TRUE)` to silence this warning
This warning is displayed only once per session.”


# Source:   lazy query [?? x 3]
# Database: postgres 12.0.11 [@localhost:5432/knock100]
   customer_id    amount relative_pos
   <chr>           <dbl>        <dbl>
 1 CS001311000059   2302      0.0970 
 2 CS004614000122    248      0.00773
 3 CS003512000043    298      0.00991
 4 CS011615000061    246      0.00765
 5 CS029212000033   3604      0.154  
 6 CS007515000119   7157      0.308  
 7 CS034515000123   3699      0.158  
 8 CS004315000058    490      0.0182 
 9 CS026414000014   6671      0.287  
10 CS001615000099    768      0.0303 

### R-061:
レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を常用対数化（底10）して顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [20]:
library(stringr)
re %>%
filter(!str_detect(customer_id, '^Z')) %>%
group_by(customer_id) %>%
summarise(amount = sum(amount, na.rm = TRUE )) %>%
mutate(log_amount = log10(amount)) %>%
head

# Source:   lazy query [?? x 3]
# Database: postgres 12.0.11 [@localhost:5432/knock100]
  customer_id    amount log_amount
  <chr>           <dbl>      <dbl>
1 CS001311000059   2302       3.36
2 CS004614000122    248       2.39
3 CS003512000043    298       2.47
4 CS011615000061    246       2.39
5 CS029212000033   3604       3.56
6 CS007515000119   7157       3.85

### R-062:
レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を自然対数化（底e）して顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [21]:
library(stringr)
re %>%
filter(!str_detect(customer_id, '^Z')) %>%
group_by(customer_id) %>%
summarise(amount = sum(amount, na.rm = TRUE )) %>%
mutate(log_amount = log(amount)) %>%
head

# Source:   lazy query [?? x 3]
# Database: postgres 12.0.11 [@localhost:5432/knock100]
  customer_id    amount log_amount
  <chr>           <dbl>      <dbl>
1 CS001311000059   2302       7.74
2 CS004614000122    248       5.51
3 CS003512000043    298       5.70
4 CS011615000061    246       5.51
5 CS029212000033   3604       8.19
6 CS007515000119   7157       8.88

### R-063:
商品データ（df_product）の単価（unit_price）と原価（unit_cost）から各商品の利益額を算出し、結果を10件表示せよ。

In [22]:
product = tbl(con, 'product')
product %>%
transmute(product_cd, unit_price, unit_cost, unit_profit = unit_price - unit_cost) %>%
head

# Source:   lazy query [?? x 4]
# Database: postgres 12.0.11 [@localhost:5432/knock100]
  product_cd unit_price unit_cost unit_profit
  <chr>           <int>     <int>       <int>
1 P040101001        198       149          49
2 P040101002        218       164          54
3 P040101003        230       173          57
4 P040101004        248       186          62
5 P040101005        268       201          67
6 P040101006        298       224          74

### R-064:
商品データ（df_product）の単価（unit_price）と原価（unit_cost）から、各商品の利益率の全体平均を算出せよ。ただし、単価と原価には欠損が生じていることに注意せよ。

In [23]:
product %>%
mutate(avg_profit_rate = mean(unit_price - unit_cost, na.rm = TRUE)/mean(unit_price)) %>%
summarise(avg_profit_rate = min(avg_profit_rate))

Warning message:
“Missing values are always removed in SQL.
Use `MIN(x, na.rm = TRUE)` to silence this warning
This warning is displayed only once per session.”


# Source:   lazy query [?? x 1]
# Database: postgres 12.0.11 [@localhost:5432/knock100]
  avg_profit_rate
            <dbl>
1           0.249

### R-065:
商品データ（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。ただし、1円未満は切り捨てること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

In [24]:
product %>%
mutate(target_price = trunc(unit_cost/0.7)) %>%
select(product_cd, unit_price, unit_cost, target_price) %>%
mutate(check_rate = (target_price - unit_cost)/target_price) %>%
head

# Source:   lazy query [?? x 5]
# Database: postgres 12.0.11 [@localhost:5432/knock100]
  product_cd unit_price unit_cost target_price check_rate
  <chr>           <int>     <int>        <dbl>      <dbl>
1 P040101001        198       149          212      0.297
2 P040101002        218       164          234      0.299
3 P040101003        230       173          247      0.300
4 P040101004        248       186          265      0.298
5 P040101005        268       201          287      0.300
6 P040101006        298       224          320      0.3  

### R-066:
商品データ（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を丸めること（四捨五入または偶数への丸めで良い）。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

In [25]:
product %>%
mutate(target_price = round(unit_cost/0.7)) %>%
select(product_cd, unit_price, unit_cost, target_price) %>%
mutate(check_rate = (target_price - unit_cost)/target_price) %>%
head

# Source:   lazy query [?? x 5]
# Database: postgres 12.0.11 [@localhost:5432/knock100]
  product_cd unit_price unit_cost target_price check_rate
  <chr>           <int>     <int>        <dbl>      <dbl>
1 P040101001        198       149          213      0.300
2 P040101002        218       164          234      0.299
3 P040101003        230       173          247      0.300
4 P040101004        248       186          266      0.301
5 P040101005        268       201          287      0.300
6 P040101006        298       224          320      0.3  

### R-067:
商品データ（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を切り上げること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

In [26]:
product %>%
mutate(target_price = ceiling(unit_cost/0.7)) %>%
select(product_cd, unit_price, unit_cost, target_price) %>%
mutate(check_rate = (target_price - unit_cost)/target_price) %>%
head

# Source:   lazy query [?? x 5]
# Database: postgres 12.0.11 [@localhost:5432/knock100]
  product_cd unit_price unit_cost target_price check_rate
  <chr>           <int>     <int>        <dbl>      <dbl>
1 P040101001        198       149          213      0.300
2 P040101002        218       164          235      0.302
3 P040101003        230       173          248      0.302
4 P040101004        248       186          266      0.301
5 P040101005        268       201          288      0.302
6 P040101006        298       224          320      0.3  

### R-068:
商品データ（df_product）の各商品について、消費税率10％の税込み金額を求めよ。1円未満の端数は切り捨てとし、結果を10件表示せよ。ただし、単価（unit_price）には欠損が生じていることに注意せよ。

In [27]:
product %>%
mutate(price_taxin = trunc(1.1 * unit_price)) %>%
select(unit_price, price_taxin) %>%
head

# Source:   lazy query [?? x 2]
# Database: postgres 12.0.11 [@localhost:5432/knock100]
  unit_price price_taxin
       <int>       <dbl>
1        198         217
2        218         239
3        230         253
4        248         272
5        268         294
6        298         327

### R-069:
レシート明細データ（df_receipt）と商品データ（df_product）を結合し、顧客毎に全商品の売上金額合計と、カテゴリ大区分コード（category_major_cd）が"07"（瓶詰缶詰）の売上金額合計を計算の上、両者の比率を求めよ。抽出対象はカテゴリ大区分コード"07"（瓶詰缶詰）の売上実績がある顧客のみとし、結果を10件表示せよ。

In [28]:
re %>%
inner_join(product, by = 'product_cd') %>% 
group_by(customer_id) %>%
summarise(amount_canned = sum(ifelse(category_major_cd == '07',amount, 0), na.rm = TRUE ), amount_total = sum(amount)) %>%
filter(amount_canned > 0) %>%
mutate(canned_ratio = as.numeric(amount_canned) / amount_total) %>%
head(10)

# Source:   lazy query [?? x 4]
# Database: postgres 12.0.11 [@localhost:5432/knock100]
   customer_id    amount_canned amount_total canned_ratio
   <chr>                  <dbl>        <dbl>        <dbl>
 1 CS001113000004          1298         1298        1    
 2 CS001114000005           486          626        0.776
 3 CS001115000010          2694         3044        0.885
 4 CS001205000004           346         1988        0.174
 5 CS001205000006          2004         3337        0.601
 6 CS001212000027           200          448        0.446
 7 CS001212000031           296          296        1    
 8 CS001212000046           108          228        0.474
 9 CS001212000070           308          456        0.675
10 CS001213000018           145          243        0.597

### R-070:
レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過日数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。

In [29]:
cu = tbl(con,'customer')

compare_date <- cu %>%
select(customer_id, application_date) %>%
inner_join(select(re, customer_id, sales_ymd), by='customer_id') %>%
transmute(customer_id, application_date = as.Date(application_date), sales_ymd = as.Date(as.character(sales_ymd)))

compare_date %>%
mutate(elapsed_days = as.Date(as.character(sales_ymd))-as.Date(application_date)) %>%
head

# Source:   lazy query [?? x 4]
# Database: postgres 12.0.11 [@localhost:5432/knock100]
  customer_id    application_date sales_ymd  elapsed_days
  <chr>          <date>           <date>            <int>
1 CS006214000001 2015-02-01       2018-11-03         1371
2 CS008415000097 2015-03-22       2018-11-18         1337
3 CS028414000014 2015-07-11       2017-07-12          732
4 CS025415000050 2016-01-31       2018-08-21          933
5 CS003515000195 2015-03-06       2019-06-05         1552
6 CS024514000042 2015-10-10       2018-12-05         1152

### R-071:
レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過月数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1ヶ月未満は切り捨てること。

In [30]:
# work in local
library(lubridate)
rel = DBI::dbReadTable(con, "receipt")
cul = DBI::dbReadTable(con, "customer")

compare_date_local <- cul %>%
select(customer_id, application_date) %>%
inner_join(select(rel, customer_id, sales_ymd), by='customer_id') %>%
transmute(customer_id, sales_ymd = ymd(sales_ymd), application_date = ymd(application_date))

compare_date_local %>% 
mutate(elapsed_months = interval(application_date, sales_ymd) %/% months(1)) %>%
head


Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union




,customer_id,sales_ymd,application_date,elapsed_months
,<chr>,<date>,<date>,<dbl>
1,CS031415000172,2017-05-07,2015-05-29,23
2,CS031415000172,2017-10-26,2015-05-29,28
3,CS031415000172,2019-03-25,2015-05-29,45
4,CS031415000172,2017-01-11,2015-05-29,19
5,CS031415000172,2019-03-25,2015-05-29,45
6,CS031415000172,2019-01-22,2015-05-29,43


### R-072:
レシート明細データ（df_receipt）の売上日（df_customer）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過年数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1年未満は切り捨てること。

In [31]:
# work in local
compare_date_local %>%
mutate(elapsed_years = interval(application_date, sales_ymd) %/% years(1)) %>%
head

,customer_id,sales_ymd,application_date,elapsed_years
,<chr>,<date>,<date>,<dbl>
1,CS031415000172,2017-05-07,2015-05-29,1
2,CS031415000172,2017-10-26,2015-05-29,2
3,CS031415000172,2019-03-25,2015-05-29,3
4,CS031415000172,2017-01-11,2015-05-29,1
5,CS031415000172,2019-03-25,2015-05-29,3
6,CS031415000172,2019-01-22,2015-05-29,3


### R-073:
レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からのエポック秒による経過時間を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。なお、時間情報は保有していないため各日付は0時0分0秒を表すものとする。

In [32]:
# work in local
compare_date_local %>%
mutate(elapsed_epoch = (sales_ymd - application_date) %>% as.numeric(units = 'secs')) %>%
head

,customer_id,sales_ymd,application_date,elapsed_epoch
,<chr>,<date>,<date>,<dbl>
1,CS031415000172,2017-05-07,2015-05-29,61257600
2,CS031415000172,2017-10-26,2015-05-29,76118400
3,CS031415000172,2019-03-25,2015-05-29,120614400
4,CS031415000172,2017-01-11,2015-05-29,51235200
5,CS031415000172,2019-03-25,2015-05-29,120614400
6,CS031415000172,2019-01-22,2015-05-29,115257600


### R-074:
レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、当該週の月曜日からの経過日数を計算し、売上日、直前の月曜日付とともに10件表示せよ（sales_ymdは数値でデータを保持している点に注意）。

In [33]:
library(lubridate)

re %>%
data.frame %>%
mutate(
	sales_ymd_d = sales_ymd %>% ymd,
	elapsed_weekdays = sales_ymd_d %>% wday(week_start = 1)-1,
	last_monday = sales_ymd_d - elapsed_weekdays
	) %>%
	head

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount,sales_ymd_d,elapsed_weekdays,last_monday
,<int>,<int>,<chr>,<int>,<int>,<chr>,<chr>,<int>,<int>,<date>,<dbl>,<date>
1,20181103,1541203200,S14006,112,1,CS006214000001,P070305012,1,158,2018-11-03,5,2018-10-29
2,20181118,1542499200,S13008,1132,2,CS008415000097,P070701017,1,81,2018-11-18,6,2018-11-12
3,20170712,1499817600,S14028,1102,1,CS028414000014,P060101005,1,170,2017-07-12,2,2017-07-10
4,20190205,1549324800,S14042,1132,1,ZZ000000000000,P050301001,1,25,2019-02-05,1,2019-02-04
5,20180821,1534809600,S14025,1102,2,CS025415000050,P060102007,1,90,2018-08-21,1,2018-08-20
6,20190605,1559692800,S13003,1112,1,CS003515000195,P050102002,1,138,2019-06-05,2,2019-06-03


In [34]:
library(lubridate)
re %>%
data.frame %>%
mutate(
	sales_ymd_d = sales_ymd %>% ymd,
	elapsed_weekdays = sales_ymd_d %>% wday(week_start = 1)-1,
	last_monday = sales_ymd_d - elapsed_weekdays
	) %>%
	head

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount,sales_ymd_d,elapsed_weekdays,last_monday
,<int>,<int>,<chr>,<int>,<int>,<chr>,<chr>,<int>,<int>,<date>,<dbl>,<date>
1,20181103,1541203200,S14006,112,1,CS006214000001,P070305012,1,158,2018-11-03,5,2018-10-29
2,20181118,1542499200,S13008,1132,2,CS008415000097,P070701017,1,81,2018-11-18,6,2018-11-12
3,20170712,1499817600,S14028,1102,1,CS028414000014,P060101005,1,170,2017-07-12,2,2017-07-10
4,20190205,1549324800,S14042,1132,1,ZZ000000000000,P050301001,1,25,2019-02-05,1,2019-02-04
5,20180821,1534809600,S14025,1102,2,CS025415000050,P060102007,1,90,2018-08-21,1,2018-08-20
6,20190605,1559692800,S13003,1112,1,CS003515000195,P050102002,1,138,2019-06-05,2,2019-06-03


### R-075:
顧客データ（df_customer）からランダムに1%のデータを抽出し、先頭から10件表示せよ。

In [35]:
cul %>% sample_frac(0.01) %>%head

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
,<chr>,<chr>,<int>,<chr>,<date>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>
1,CS023615000070,玉城 美佳,1,女性,1951-08-06,67,144-0055,東京都大田区仲六郷**********,S14023,20150813,C-20100712-A
2,CS005412000024,杉山 まみ,1,女性,1977-01-14,42,176-0002,東京都練馬区桜台**********,S13005,20150808,2-20080407-1
3,CS031513000201,松井 佳乃,1,女性,1962-04-17,56,151-0061,東京都渋谷区初台**********,S13031,20150508,4-20090615-6
4,CS001615000271,杉野 奈央,1,女性,1954-01-06,65,144-0035,東京都大田区南蒲田**********,S13001,20160714,1-20091130-1
5,CS014515000252,野沢 千佳子,1,女性,1965-06-08,53,264-0032,千葉県千葉市若葉区みつわ台**********,S12014,20180510,0-00000000-0
6,CS023414000087,梅本 明日,1,女性,1969-06-24,49,212-0021,神奈川県川崎市幸区都町**********,S14023,20150929,E-20101029-F


### R-076:
顧客データ（df_customer）から性別コード（gender_cd）の割合に基づきランダムに10%のデータを層化抽出し、性別コードごとに件数を集計せよ。

### R-077:
レシート明細データ（df_receipt）の売上金額を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。なお、外れ値は売上金額合計を対数化したうえで平均と標準偏差を計算し、その平均から3σを超えて離れたものとする（自然対数と常用対数のどちらでも可）。結果は10件表示せよ。


In [36]:
re %>%
group_by(customer_id) %>%
summarise(log_amount = log(sum(amount, na.rm = TRUE))) %>%
mutate(mean = mean(log_amount, na.rm = TRUE), sd = sd(log_amount,na.rm = TRUE)) %>%
filter(!(log_amount <= mean+3*sd)) %>%
head

# Source:   lazy query [?? x 4]
# Database: postgres 12.0.11 [@localhost:5432/knock100]
  customer_id    log_amount  mean    sd
  <chr>               <dbl> <dbl> <dbl>
1 ZZ000000000000       16.3  7.27  1.14

### R-078:
レシート明細データ（df_receipt）の売上金額（amount）を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、ここでは外れ値を第1四分位と第3四分位の差であるIQRを用いて、「第1四分位数-1.5×IQR」を下回るもの、または「第3四分位数+1.5×IQR」を超えるものとする。結果は10件表示せよ。

In [37]:
# assume you are on a local machine
library(stringr)

re %>%
data.frame %>%
filter(!str_detect(customer_id, '^Z')) %>%
group_by(customer_id) %>%
summarise(amount = sum(amount, na.rm = TRUE)) %>%
mutate(iqr = quantile(amount)[4] - quantile(amount)[2]) %>%
mutate(thre_lower = quantile(amount)[2] - 1.5*iqr, thre_upper = quantile(amount)[4] + 1.5*iqr) %>%
filter(amount < thre_lower | thre_upper < amount) %>%
head

customer_id,amount,iqr,thre_lower,thre_upper
<chr>,<int>,<dbl>,<dbl>,<dbl>
CS001414000048,8584,3101.5,-4104,8302
CS001605000009,18925,3101.5,-4104,8302
CS002415000594,9568,3101.5,-4104,8302
CS004414000181,9584,3101.5,-4104,8302
CS005415000137,8734,3101.5,-4104,8302
CS006414000001,9156,3101.5,-4104,8302


In [38]:
# assume you are on a local machine
rel = DBI::dbReadTable(con, "receipt")
library(stringr)

rel %>%
filter(!str_detect(customer_id, '^Z')) %>%
group_by(customer_id) %>%
summarise(amount = sum(amount, na.rm = TRUE)) %>%
mutate(iqr = quantile(amount)[4] - quantile(amount)[2]) %>%
mutate(thre_lower = quantile(amount)[2] - 1.5*iqr, thre_upper = quantile(amount)[4] + 1.5*iqr) %>%
filter(amount < thre_lower | thre_upper < amount) %>%
head

customer_id,amount,iqr,thre_lower,thre_upper
<chr>,<int>,<dbl>,<dbl>,<dbl>
CS001414000048,8584,3101.5,-4104,8302
CS001605000009,18925,3101.5,-4104,8302
CS002415000594,9568,3101.5,-4104,8302
CS004414000181,9584,3101.5,-4104,8302
CS005415000137,8734,3101.5,-4104,8302
CS006414000001,9156,3101.5,-4104,8302


### R-079:
商品データ（df_product）の各項目に対し、欠損数を確認せよ。

In [39]:
pr %>%
data.frame %>%
summarise(across(everything(), ~ is.na(.) %>% sum()))

ERROR: Error in data.frame(.): object 'pr' not found


### R-080:
商品データ（df_product）のいずれかの項目に欠損が発生しているレコードを全て削除した新たな商品データを作成せよ。なお、削除前後の件数を表示させ、079で確認した件数だけ減少していることも確認すること。

In [ ]:
# complete.cases does not works on tbl
pr %>% data.frame %>%
filter(complete.cases(.)) %>% nrow

[1] 10023

### R-081:
単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの平均値で補完した新たな商品データを作成せよ。なお、平均値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。

In [ ]:
library(tidymodels)

pr %>%
data.frame %>%
recipe() %>%
step_impute_mean(unit_price, unit_cost) %>%
prep() %>%
juice() %>%
head
# filter(unit_cost %>% is.na)

product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost
<fct>,<fct>,<fct>,<fct>,<int>,<int>
P040101001,04,0401,040101,198,149
P040101002,04,0401,040101,218,164
P040101003,04,0401,040101,230,173
P040101004,04,0401,040101,248,186
P040101005,04,0401,040101,268,201
P040101006,04,0401,040101,298,224


### R-082:
単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの中央値で補完した新たな商品データを作成せよ。なお、中央値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。

In [ ]:
pr %>%
data.frame %>%
recipe() %>%
step_impute_median(unit_price, unit_cost) %>%
prep() %>%
juice() %>%
#head
filter(unit_cost %>% is.na) %>%head

product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost
<fct>,<fct>,<fct>,<fct>,<int>,<int>


In [ ]:
pr %>%
  replace_na(list(unit_price = mean(.$unit_price, na.rm = T) %>% round, unit_cost = mean(.$unit_cost, na.rm = T) %>% round))

Warning message in mean.default(.$unit_price, na.rm = T):
“argument is not numeric or logical: returning NA”
Warning message in mean.default(.$unit_cost, na.rm = T):
“argument is not numeric or logical: returning NA”


# Source:   lazy query [?? x 6]
# Database: postgres 12.0.9 [@localhost:5432/knock100]
   product_cd category_major_cd category_medium_cd category_small_cd unit_price
   <chr>      <chr>             <chr>              <chr>                  <int>
 1 P040101001 04                0401               040101                   198
 2 P040101002 04                0401               040101                   218
 3 P040101003 04                0401               040101                   230
 4 P040101004 04                0401               040101                   248
 5 P040101005 04                0401               040101                   268
 6 P040101006 04                0401               040101                   298
 7 P040101007 04                0401               040101                   338
 8 P040101008 04                0401               040101                   420
 9 P040101009 04                0401               040101                   498
10 P040101010 04                0

### R-083:
単価（unit_price）と原価（unit_cost）の欠損値について、各商品のカテゴリ小区分コード（category_small_cd）ごとに算出した中央値で補完した新たな商品データを作成せよ。なお、中央値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。

In [ ]:
pr %>%
group_by(category_small_cd) %>%
summarise(
		price_median = unit_price %>% median(na.rm = TRUE) %>% round(),
		cost_median = unit_cost %>% median(na.rm = TRUE) %>% round()
		) %>%
inner_join(pr, by='category_small_cd') %>%
mutate(
	unit_price = ifelse(unit_price %>% is.na, price_median, unit_price),
	unit_cost = ifelse(unit_cost %>% is.na, cost_median, unit_cost)
)

# Source:   lazy query [?? x 8]
# Database: postgres 12.0.9 [@localhost:5432/knock100]
   category_small_cd price_median cost_median product_cd category_major_cd
   <chr>                    <dbl>       <dbl> <chr>      <chr>            
 1 040101                     283         213 P040101001 04               
 2 040101                     283         213 P040101002 04               
 3 040101                     283         213 P040101003 04               
 4 040101                     283         213 P040101004 04               
 5 040101                     283         213 P040101005 04               
 6 040101                     283         213 P040101006 04               
 7 040101                     283         213 P040101007 04               
 8 040101                     283         213 P040101008 04               
 9 040101                     283         213 P040101009 04               
10 040101                     283         213 P040101010 04               
# … with more

### R-084:
顧客データ（df_customer）の全顧客に対して全期間の売上金額に占める2019年売上金額の割合を計算し、新たなデータを作成せよ。ただし、売上実績がない場合は0として扱うこと。そして計算した割合が0超のものを抽出し、結果を10件表示せよ。また、作成したデータに欠損が存在しないことを確認せよ。

In [ ]:
library(lubridate)
cu = tbl(con, 'customer')

re %>%
data.frame %>%
mutate(year = sales_ymd %>% ymd %>% year) %>%
group_by(customer_id) %>%
summarize(
	amount_all = sum(amount, na.rm = TRUE),
	amount_2019 = sum(ifelse(year == 2019,amount,0 ), na.rm = TRUE)
) %>%
right_join(cu %>% data.frame, by = 'customer_id') %>%
replace_na(list(amount_all = 0, amount_2019 = 0)) %>%
filter(amount_2019 > 0) %>% head(10)

customer_id,amount_all,amount_2019,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
<chr>,<int>,<dbl>,<chr>,<int>,<chr>,<date>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>
CS001113000004,1298,1298,葛西 莉央,1,女性,2003-02-22,16,144-0056,東京都大田区西六郷**********,S13001,20151105,7-20100308-1
CS001114000005,626,188,安 里穂,1,女性,2004-11-22,14,144-0056,東京都大田区西六郷**********,S13001,20160412,5-20100731-7
CS001115000010,3044,578,藤沢 涼,1,女性,2006-05-16,12,144-0056,東京都大田区西六郷**********,S13001,20150417,B-20100405-9
CS001205000004,1988,702,奥山 秀隆,0,男性,1993-02-28,26,144-0056,東京都大田区西六郷**********,S13001,20160615,9-20100625-C
CS001205000006,3337,486,福士 明,0,男性,1993-06-12,25,144-0056,東京都大田区西六郷**********,S13001,20161111,B-20100224-C
CS001211000025,456,456,河野 夏希,1,女性,1996-06-09,22,140-0013,東京都品川区南大井**********,S13001,20160505,4-20100322-2
CS001212000070,456,456,浜本 愛,1,女性,1996-01-14,23,210-0831,神奈川県川崎市川崎区観音**********,S13001,20160918,4-20101018-2
CS001214000009,4685,664,平尾 奈月,1,女性,1990-05-02,28,144-0056,東京都大田区西六郷**********,S13001,20160108,D-20100902-D
CS001214000017,4132,2962,小野 寿々花,1,女性,1996-06-05,22,144-0056,東京都大田区西六郷**********,S13001,20150409,C-20101006-C


### R-085:
顧客データ（df_customer）の全顧客に対し、郵便番号（postal_cd）を用いてジオコードデータ（df_geocode）を紐付け、新たな顧客データを作成せよ。ただし、1つの郵便番号（postal_cd）に複数の経度（longitude）、緯度（latitude）情報が紐づく場合は、経度（longitude）、緯度（latitude）の平均値を算出して使用すること。また、作成結果を確認するために結果を10件表示せよ。

### R-086:
085で作成した緯度経度つき顧客データに対し、会員申込店舗コード（application_store_cd）をキーに店舗データ（df_store）と結合せよ。そして申込み店舗の緯度（latitude）・経度情報（longitude)と顧客住所（address）の緯度・経度を用いて申込み店舗と顧客住所の距離（単位：km）を求め、顧客ID（customer_id）、顧客住所（address）、店舗住所（address）とともに表示せよ。計算式は以下の簡易式で良いものとするが、その他精度の高い方式を利用したライブラリを利用してもかまわない。結果は10件表示せよ。

### R-087:
顧客データ（df_customer）では、異なる店舗での申込みなどにより同一顧客が複数登録されている。名前（customer_name）と郵便番号（postal_cd）が同じ顧客は同一顧客とみなして1顧客1レコードとなるように名寄せした名寄顧客データを作成し、顧客データの件数、名寄顧客データの件数、重複数を算出せよ。ただし、同一顧客に対しては売上金額合計が最も高いものを残し、売上金額合計が同一もしくは売上実績がない顧客については顧客ID（customer_id）の番号が小さいものを残すこととする。

### R-088:
087で作成したデータを元に、顧客データに統合名寄IDを付与したデータを作成せよ。ただし、統合名寄IDは以下の仕様で付与するものとする。
- 重複していない顧客：顧客ID（customer_id）を設定
- 重複している顧客：前設問で抽出したレコードの顧客IDを設定

顧客IDのユニーク件数と、統合名寄IDのユニーク件数の差も確認すること。

### R-089:
売上実績がある顧客を、予測モデル構築のため学習用データとテスト用データに分割したい。それぞれ8:2の割合でランダムにデータを分割せよ。

### R-090:
レシート明細データ（df_receipt）は2017年1月1日〜2019年10月31日までのデータを有している。売上金額（amount）を月次で集計し、学習用に12ヶ月、テスト用に6ヶ月の時系列モデル構築用データを3セット作成せよ。